In [1]:
import os
import pandas as pd
import numpy as np
import preprocessor as prep
import postprocessor as postp
import processor as proc
import pandas as pd
import unifloc.uniflocpy.uTools.plotly_workflow as pw
import datetime
from sklearn import metrics
import multiprocessing as mp
from multiprocessing import Pool
from preprocessor import load_and_edit_chess_data
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from pandas.api.types import is_numeric_dtype

C:\Users\LemikhovAlex\Documents\CET\git\unifloc_vba\\
Затрачено времени всего: 0.0


Немножко пошалим, тупо запустим все скважины, про которые есть `%well_name%.csv` чтобы протестить работу с шахматками. Не надо забывать закрывать всплывающие вкладки, они убьют компуцкер

In [2]:
#well_names = ['569', '601', '1354', '1567', '1602', '1628']
well_names = ['1628']
for well_name in well_names:
    chess_file_name = 'Скв. ' + well_name + ' (01.08.2018-28.02.2019).xls'
    tr_name = "Техрежим, , февраль 2019.xls"
    # TODO убрать флаги
    read_initial_data = True
    plot_initial_data = True
    create_input_data = True
    multiprocessing_on = True #TODO может быть убрать флаги, как лучше?
    created_input_data_type = 0 # 0 - adaptation input, 1 - restore input with calibation coefficients

    current_path = os.getcwd()
    time_mark = ''  # datetime.datetime.today().strftime('%Y_%m_%d_%H_%M_%S')
    path_to_data = current_path + "\\data\\"
    path_to_work_dir = current_path + "\\data\\" + well_name +  "\\"
    save_dir_name = 'init_edit'
    path_to_save = path_to_work_dir + save_dir_name + '\\'

    cs_data_filename = path_to_save + well_name + "_first_edit.csv"

    left_boundary = [datetime.datetime(2019,1,1)]#, datetime.datetime(2018,11,29)]
    right_boundary = [datetime.datetime(2019,2,28)]#, datetime.datetime(2019,2,28)]


    if read_initial_data:
        try:
            os.mkdir(path_to_work_dir + save_dir_name)
        except:
            pass
        data_file_path = path_to_work_dir + well_name + ".csv"
        try:
            try:
                well_data = pd.read_csv(data_file_path,sep=';', header=None, skiprows = 1)
            except:
                well_data = pd.read_csv(data_file_path,sep='\\t', header=None)
            well_data = prep.initial_editing(well_data, well_name)
            well_data = prep.create_edited_df(well_data)
        except:
            data_file_path = path_to_work_dir + well_name + ".csv"
            well_data = pd.read_csv(data_file_path, sep='\\t', skiprows = 1, skipfooter =1, index_col = 'Названия строк')#, header=None)
            well_data.index.name = 'Время'
            well_data.index = pd.to_datetime(well_data.index)
            del well_data['Общий итог']
            well_data.columns = prep.del_inf_in_columns_name(well_data, well_name)

    #TODO использовать библиотеку modin вместо pandas
    well_data.to_csv(path_to_save + well_name + "_first_edit.csv")

    well_data['ГФ, м3/м3'] = well_data['Объемный дебит газа'] / well_data['Объемный дебит нефти']
    qliq = {'Объемный дебит жидкости':['Объемный дебит жидкости']}
    gor = {'ГФ, м3/м3':['ГФ, м3/м3']}
    wc = {'Процент обводненности':['Процент обводненности']}
    pressure_intake = {'Давление на приеме': ['Давление на приеме насоса (пласт. жидкость)']}
    pressure_wh = {'Линейное давление': ['Линейное давление']}
    temp_intake = {'Температура на приеме насоса': ['Температура на приеме насоса (пласт. жидкость)']}
    frequencies = {'Выходная частота ПЧ':
                   ['Выходная частота ПЧ']}
    load = {'Загрузка двигателя': ['Загрузка двигателя']}
    power = {'Активная мощность':['Активная мощность']}
    voltage = {'Напряжение на выходе ТМПН':['Напряжение на выходе ТМПН']}
    cos = {'Коэффициент мощности':['Коэффициент мощности']}
    time = {'Время замеров':['Время замера фактическое','Время замера плановое']}
    current =  {'Ток фазы А':['Ток фазы А']}
    all_banches = [qliq,gor, wc, time, pressure_intake, pressure_wh, 
                   temp_intake, frequencies, load,power, voltage,cos,current]

    pw.create_report_html(well_data, all_banches, path_to_save + well_name + "_first_edit_report.html")
    del well_data['ГФ, м3/м3']

    chess_data = prep.load_and_edit_chess_data(path_to_work_dir + chess_file_name,
                                         '1d', without_changing = True)
    qliq = {'Qж ТМ':['Qж ТМ']}
    wc = {'Обв ТМ':['Обв ТМ']}
    pressure_intake = {'Рэцн ТМ': ['Рэцн ТМ']}
    pressure_wh = {'Рлин ТМ': ['Рлин ТМ']}
    pressure_buf = {'Р буферное': ['Рбуф ТМ', 'Рбуф']}
    temp_intake = {'Темп. ж. ТМ ': ['Темп. ж. ТМ']}
    frequencies = {'F вращ ТМ':
                   ['F вращ ТМ']
                  }
    load = {'Загр. ПЭД (ТМ)': ['Загр. ПЭД (ТМ)']}
    cos = {'Cos ф (ТМ)':['Cos ф (ТМ)']}
    choke = {'Dшт':['Dшт']}
    current =  {'I A ТМ':['I A ТМ']}
    all_banches = [qliq, wc, pressure_intake, pressure_wh, pressure_buf,
                   temp_intake, frequencies, load,cos,choke, current]

    pw.create_report_html(chess_data, all_banches, path_to_save + well_name + "_chess_report.html")
    print(f'{well_name} done')

1628 done
